In [10]:
import os
import pandas as pd
import numpy as np
import math

In [11]:
def colunm_change(str_):
    if 'YR' in str_:
        return str_[2:]
    else:
        return str_

def table_name(file_name):
    aux = file_name.split("_")[-1]
    str_ = aux.split('.')[:-1]
    return "_".join(str_)

In [12]:
files = os.listdir('data')

out_df = pd.DataFrame()
for file in files:
    df_aux = pd.read_csv('data/'+file)
    df_aux.columns = [colunm_change(c) for c in df_aux.columns]

    out = pd.melt(df_aux, id_vars=['economy',], value_vars=None,
                var_name='ano', value_name=table_name(file))
    

    if(len(out_df) == 0):
        out_df = out
    else:
        out_df = out_df.merge(out)

out_df

,economy,ano,AG_AGR_TRAC_NO,AG_LND_AGRI_ZS,EN_ATM_CO2E_PC,NY_GDP_MKTP_KD_ZG,NY_GDP_PCAP_PP_KD,EG_USE_COMM_CL_ZS,EG_USE_COMM_FO_ZS,AG_LND_FRST_ZS,EG_ELC_FOSL_ZS
0,ABW,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFE,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17019,XKX,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17020,YEM,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17021,ZAF,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17022,ZMB,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
economias = out_df['economy'].unique()
anos = out_df['ano'].unique()
tables_convert = ['AG_AGR_TRAC_NO', 'NY_GDP_PCAP_PP_KD', 'EG_USE_COMM_FO_ZS', 'EN_ATM_CO2E_PC'] #precisa converter para porcentagem e verificar o aumento
tables_dif = ['EG_ELC_FOSL_ZS', 'AG_LND_AGRI_ZS',  'NY_GDP_MKTP_KD_ZG', 'AG_LND_FRST_ZS',  'EG_USE_COMM_CL_ZS']


out_df.loc[out_df['economy'] == economias[0], tables_convert]

,AG_AGR_TRAC_NO,NY_GDP_PCAP_PP_KD,EG_USE_COMM_FO_ZS,EN_ATM_CO2E_PC
0,NaN,NaN,NaN,NaN
266,NaN,NaN,NaN,NaN
532,NaN,NaN,NaN,NaN
798,NaN,NaN,NaN,NaN
1064,NaN,NaN,NaN,NaN
...,...,...,...,...
15694,NaN,38511.336876,NaN,NaN
15960,NaN,29236.047264,NaN,NaN
16226,NaN,37333.515685,NaN,NaN
16492,NaN,41273.615055,NaN,NaN


In [14]:
def get_percent_gross(list_):
    for i in range(len(list_)-1, 1, -1):

        if not math.isnan(float(list_[i])) and not math.isnan(float(list_[i-1])): #os valores não são nan
            try:
                list_[i] = (list_[i]/list_[i-1] - 1) * 100 #porcentagem de crescimento
            except:
                if list_[i] > 0:
                    list_[i] = 100 #o ultimo valor foi 100 então ele deve ser 100% de crescimento
                else:
                    list_[i] = 0
    
    for i in range(len(list_)):
        if not math.isnan(list_[i]): #tornar o primeiro valor em 0
            list_[i] = 0
            break
            
    return list_

def get_percent_dif(list_):
    for i in range(len(list_)-1, 1, -1):
        if not math.isnan(float(list_[i])) and not math.isnan(float(list_[i-1])): #os valores não são nan
            try:
                list_[i] = list_[i] - list_[i-1] #diferenca entre porcentagens
            except:
                if list_[i] > 0:
                    list_[i] = 100 #o ultimo valor foi 100 então ele deve ser 100% de crescimento
                else:
                    list_[i] = 0
    
    for i in range(len(list_)):
        if not math.isnan(list_[i]): #tornar o primeiro valor em 0
            list_[i] = 0
            break
    
    return list_
            

In [15]:

for e in economias:
    for i in tables_convert: #remove ano
        lista_aux = out_df.loc[out_df['economy'] == e, i]
        lista_aux = list(lista_aux)
        lista_aux = get_percent_gross(lista_aux)
        lista_aux = lista_aux
        # print(len(lista_aux))
        out_df.loc[out_df['economy'] == e, i] = lista_aux

for e in economias:
    for i in tables_dif:
        lista_aux = out_df.loc[out_df['economy'] == e, i] #pega a coluna
        lista_aux = list(lista_aux)
        lista_aux = get_percent_dif(lista_aux)
        lista_aux = lista_aux
        # print(len(lista_aux))
        out_df.loc[out_df['economy'] == e, i] = lista_aux

out_df.loc[out_df['economy'] == 'BRA']

,economy,ano,AG_AGR_TRAC_NO,AG_LND_AGRI_ZS,EN_ATM_CO2E_PC,NY_GDP_MKTP_KD_ZG,NY_GDP_PCAP_PP_KD,EG_USE_COMM_CL_ZS,EG_USE_COMM_FO_ZS,AG_LND_FRST_ZS,EG_ELC_FOSL_ZS
29,BRA,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,BRA,1961,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
561,BRA,1962,13.888889,0.443520,NaN,-2.000000,NaN,NaN,NaN,NaN,NaN
827,BRA,1963,13.414634,0.443520,NaN,-6.000000,NaN,NaN,NaN,NaN,NaN
1093,BRA,1964,10.752688,0.443520,NaN,2.800000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15723,BRA,2019,NaN,0.074501,-0.653568,-0.562889,0.448280,NaN,NaN,-0.149902,NaN
15989,BRA,2020,NaN,0.074489,-5.278347,-4.497537,-3.918004,NaN,NaN,-0.141048,NaN
16255,BRA,2021,NaN,0.074476,NaN,8.265608,4.435353,NaN,NaN,-0.146951,NaN
16521,BRA,2022,NaN,NaN,NaN,-2.088319,2.428702,NaN,NaN,NaN,NaN


In [16]:
out_df.loc[out_df['economy'] == 'BRA']

,economy,ano,AG_AGR_TRAC_NO,AG_LND_AGRI_ZS,EN_ATM_CO2E_PC,NY_GDP_MKTP_KD_ZG,NY_GDP_PCAP_PP_KD,EG_USE_COMM_CL_ZS,EG_USE_COMM_FO_ZS,AG_LND_FRST_ZS,EG_ELC_FOSL_ZS
29,BRA,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,BRA,1961,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
561,BRA,1962,13.888889,0.443520,NaN,-2.000000,NaN,NaN,NaN,NaN,NaN
827,BRA,1963,13.414634,0.443520,NaN,-6.000000,NaN,NaN,NaN,NaN,NaN
1093,BRA,1964,10.752688,0.443520,NaN,2.800000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15723,BRA,2019,NaN,0.074501,-0.653568,-0.562889,0.448280,NaN,NaN,-0.149902,NaN
15989,BRA,2020,NaN,0.074489,-5.278347,-4.497537,-3.918004,NaN,NaN,-0.141048,NaN
16255,BRA,2021,NaN,0.074476,NaN,8.265608,4.435353,NaN,NaN,-0.146951,NaN
16521,BRA,2022,NaN,NaN,NaN,-2.088319,2.428702,NaN,NaN,NaN,NaN


In [17]:
out_df.loc[out_df['economy'] == 'BRA']

,economy,ano,AG_AGR_TRAC_NO,AG_LND_AGRI_ZS,EN_ATM_CO2E_PC,NY_GDP_MKTP_KD_ZG,NY_GDP_PCAP_PP_KD,EG_USE_COMM_CL_ZS,EG_USE_COMM_FO_ZS,AG_LND_FRST_ZS,EG_ELC_FOSL_ZS
29,BRA,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,BRA,1961,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
561,BRA,1962,13.888889,0.443520,NaN,-2.000000,NaN,NaN,NaN,NaN,NaN
827,BRA,1963,13.414634,0.443520,NaN,-6.000000,NaN,NaN,NaN,NaN,NaN
1093,BRA,1964,10.752688,0.443520,NaN,2.800000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15723,BRA,2019,NaN,0.074501,-0.653568,-0.562889,0.448280,NaN,NaN,-0.149902,NaN
15989,BRA,2020,NaN,0.074489,-5.278347,-4.497537,-3.918004,NaN,NaN,-0.141048,NaN
16255,BRA,2021,NaN,0.074476,NaN,8.265608,4.435353,NaN,NaN,-0.146951,NaN
16521,BRA,2022,NaN,NaN,NaN,-2.088319,2.428702,NaN,NaN,NaN,NaN


In [18]:
out_df.loc[:, 'EN_ATM_CO2E_PC'] = out_df.loc[:, 'EN_ATM_CO2E_PC'].apply(lambda x: np.nan if x == 0 else x) #transforma dados 0 em nan para remorção
out_df.dropna(subset='EN_ATM_CO2E_PC', inplace=True) #remove as linhas nulas de EN_ATM_CO2E_PC nossa label
out_df.isna().sum()

economy                 0
ano                     0
AG_AGR_TRAC_NO       4725
AG_LND_AGRI_ZS         93
EN_ATM_CO2E_PC          0
NY_GDP_MKTP_KD_ZG     292
NY_GDP_PCAP_PP_KD     449
EG_USE_COMM_CL_ZS    2720
EG_USE_COMM_FO_ZS    2688
AG_LND_FRST_ZS        103
EG_ELC_FOSL_ZS       2638
dtype: int64

In [19]:
out_df.fillna(0, inplace = True)
out_df.to_csv('final_data_clean.csv')